In [ ]:
import sys
!{sys.executable} -m pip install pandas

In [ ]:
!{sys.executable} -m pip install matplotlib

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from survival import MIRModel

In [2]:
#Compiled combined MIR dataset
#df = pd.read_csv('/home/j/temp/fed1/2019_mirs_with_background_mortality.csv')
#df = pd.read_csv('/home/j/temp/fed1/input_file_with_other_mortality_calculated_from_numbers.csv')
df = pd.read_csv('/home/j/temp/fed1/all_ages_input_file_with_other_mortality_calculated_from_numbers_20HAQIlocs.csv')

# subset the useful rows and columns for the fit
selected_columns = [
    'acause',
    'location_id',
    'year_id',
    'age_group_id',
    'sex_id',
    'mi_ratio',
    'other_mortality'
]
df_sub = df.rename(columns={'year_id':'Year'})

#remove NAs and MIR >1
df_sub = df_sub.dropna()
df_sub = df_sub[df_sub['mi_ratio']<=1]

In [3]:
df_sub.head()

,Unnamed: 0,age_group_id,age_group_name,location_id,location_name,Year,sex_id,acause,cause_name,deaths,cases,total_deaths,non_cancer_deaths,mi_ratio,population,other_mortality
0,1,22,All Ages,8,Taiwan (Province of China),1990,1,neo_esophageal,Esophageal cancer,740.575618,781.511588,63577.367257,62836.791639,0.947619,1.054138e+07,0.005961
1,2,22,All Ages,8,Taiwan (Province of China),1990,1,neo_stomach,Stomach cancer,1898.603028,2263.001821,63577.367257,61678.764229,0.838975,1.054138e+07,0.005851
2,3,22,All Ages,8,Taiwan (Province of China),1990,1,neo_liver,Liver cancer,1615.074426,1685.726925,63577.367257,61962.292830,0.958088,1.054138e+07,0.005878
3,4,22,All Ages,8,Taiwan (Province of China),1990,1,neo_liver_hepb,Liver cancer due to hepatitis B,986.454887,1051.032018,63577.367257,62590.912370,0.938558,1.054138e+07,0.005938
5,6,22,All Ages,8,Taiwan (Province of China),1990,1,neo_liver_alcohol,Liver cancer due to alcohol use,176.174825,179.777137,63577.367257,63401.192431,0.979962,1.054138e+07,0.006015


In [7]:
#df_sub_washington = df_sub[df_sub['location_id']==102]

In [4]:
#Input disease_period (n) and num_year survival desired ¶
disease_period = 5
num_years = 5

model = MIRModel(df_sub['mi_ratio'],
                 df_sub['other_mortality'],
                 disease_period=disease_period)
model.compute_excess_mortality()
survival_rate = model.get_survival_rate(num_years=num_years)

df_sub['excess_mortality'] = model.excess_mortality
df_sub['abs_survival_rate'] = survival_rate['abs']
df_sub['rel_survival_rate'] = survival_rate['rel']
df_sub.sort_values('Year', inplace=True)

# Print head of data for check
df_sub.head()

,Unnamed: 0,age_group_id,age_group_name,location_id,location_name,Year,sex_id,acause,cause_name,deaths,cases,total_deaths,non_cancer_deaths,mi_ratio,population,other_mortality,excess_mortality,abs_survival_rate,rel_survival_rate
0,1,22,All Ages,8,Taiwan (Province of China),1990,1,neo_esophageal,Esophageal cancer,740.575618,781.511588,63577.367257,62836.791639,0.947619,1.054138e+07,0.005961,0.467925,0.040309,0.041532
41873,41874,22,All Ages,142,Iran (Islamic Republic of),1990,2,neo_bone,Malignant neoplasm of bone and articular carti...,30.181422,48.062180,146492.914779,146462.733357,0.627966,2.861760e+07,0.005118,0.182242,0.354399,0.363608
41872,41873,22,All Ages,142,Iran (Islamic Republic of),1990,2,neo_tissue_sarcoma,Soft tissue and other extraosseous sarcomas,20.196488,27.278297,146492.914779,146472.718291,0.740387,2.861760e+07,0.005118,0.240774,0.243875,0.250213
41871,41872,22,All Ages,142,Iran (Islamic Republic of),1990,2,neo_eye_other,Other eye cancers,9.327043,69.099736,146492.914779,146483.587736,0.134979,2.861760e+07,0.005119,0.028896,0.841110,0.862971
41869,41870,22,All Ages,142,Iran (Islamic Republic of),1990,2,neo_eye,Eye cancer,13.623145,72.153581,146492.914779,146479.291635,0.188808,2.861760e+07,0.005119,0.041446,0.787875,0.808352


In [5]:
df_sub.to_csv('/home/j/temp/fed1/new_methods_all_ages_predictions_20HAQIlocs_using_2019_inputs_and_direct_P_o_calc_disease_pd_5_num_years_5_all_years.csv')